In [1]:
import sys
# make sure to use position 1
sys.path.insert(1, "../../")

In [2]:
import numpy as np
import tensorflow as tf
import cv2
import time
import os
import src.darknet.darknet_core as darknet_core
import src.autoencoder.autoencoder as ae

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
decoderStructure = {
    'outputImgDim':[64,64,64,1],
    'trainable':True,
    'activation':tf.nn.leaky_relu,
    'filterNumList':[512,256,128,64,1],
    'kernelSizeList':[4,4,4,4,4],
    'stridesList':[1,2,2,2,2],
    'lastLayerActivation':tf.nn.sigmoid
}

In [4]:
class nolbo(object):
    def __init__(self, inputImgSize=[416,416], latentDim=None, decoderStructure=None, batchSize=32, learningRate = 0.001):
        self._decStruct = decoderStructure
        self._darknetCore = None
        self._decoder = None
        self._inputImgSize = inputImgSize
        self._inputImages = None
        self._zs = None
        self._outputImages = None
        self._outputImagesGT = None
        self._inputImgSize = inputImgSize
        self._batchSize = batchSize
        self._lr = learningRate
        self._variables = None
        self._update_ops = None
        self._saver = None
        self._latentDim = latentDim
        self._buildNetwork()
        
    def _buildNetwork(self):
        print "build network..."
        outputImageDim = self._decStruct['outputImgDim']
        self._inputImages = tf.placeholder(tf.float32, shape=(np.concatenate([[None], self._inputImgSize, [1]])))
        self._outputImages = tf.placeholder(tf.float32, shape=(np.concatenate([[None], outputImageDim])))
        self._yolo = darknet_core.yolov2(lastLayserChannelNum=self._latentDim, nameScope='nolboEnc')
        self._decoder = ae.decoder(architecture=self._decStruct, decoderName='nolboDec')
        self._zs = self._yolo(self._inputImages)
        self._outputImages = self._decoder(self._zs[:,1,1,:])
        self._outputImages = self._decoder(self._zs[:,1,1,:])
        zsConvVariables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='nolbo_zsConv')
        zsConvUpdate_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='nolbo_zsConv')
        self._variables = self._yolo.variables + self._decoder.variables
        self._update_ops= self._yolo.update_ops + self._decoder.update_ops
        self._saver = tf.train.Saver(var_list=self._variables)
        print "done!"

In [5]:
nb = nolbo(decoderStructure=decoderStructure, latentDim=256)

build network...
(?, 416, 416, 32)
(?, 208, 208, 32)
(?, 208, 208, 64)
(?, 104, 104, 64)
(?, 104, 104, 128)
(?, 104, 104, 64)
(?, 104, 104, 128)
(?, 52, 52, 128)
(?, 52, 52, 256)
(?, 52, 52, 128)
(?, 52, 52, 256)
(?, 26, 26, 256)
(?, 26, 26, 512)
(?, 26, 26, 256)
(?, 26, 26, 512)
(?, 26, 26, 256)
(?, 26, 26, 512)
(?, 13, 13, 512)
(?, 13, 13, 1024)
(?, 13, 13, 512)
(?, 13, 13, 1024)
(?, 13, 13, 512)
(?, 13, 13, 1024)
(?, 13, 13, 256)
decoder...
(?, 1024)
(?, 4, 4, 4, 16)
(?, 4, 4, 4, 512)
(?, 8, 8, 8, 256)
(?, 16, 16, 16, 128)
(?, 32, 32, 32, 64)
(?, 64, 64, 64, 1)
decoder...
(?, 1024)
(?, 4, 4, 4, 16)
(?, 4, 4, 4, 512)
(?, 8, 8, 8, 256)
(?, 16, 16, 16, 128)
(?, 32, 32, 32, 64)
(?, 64, 64, 64, 1)
done!
